The purpose of this method is just to explore the method and consider (A) how more advanced users may interact and (B) how sensitivity testing may work

In [8]:
# Import libraries
import numpy as np
import scipy
from scipy.stats import norm, lognorm
import matplotlib.pyplot as plt
import pandas as pd


In [11]:
def calculate_cumulative_conf(areaP90: float=1., areaP10: float=10., pdP90: float=10., pdP10: float=24):
    """Calculate cumulative confidence level for expected development size in MW

    Args:
        areaP90 (float): pessimistic area in sqkm
        areaP10 (float): optimistic area in sqkm
        pdP90 (float): pessimistic power density in MWe/sqkm
        pdP10 (float): optimistic power density in MWe/sqkm

    Returns:
        prob_df (pandas Dataframe): cumulative confidence curve in Reservoir Size
    """
    # calculate area > 250 °C
    area_mu = ((np.log(areaP90)+np.log(areaP10))/2)
    area_sigma = (np.log(areaP10)-np.log(areaP90))/((norm.ppf(0.9)-(norm.ppf(0.1))))

    # calculate powerdensity mean and standard dev
    powerdens_mu = ((np.log(pdP90)+np.log(pdP10))/2)
    powerdens_sigma = (np.log(pdP10)-np.log(pdP90))/((norm.ppf(0.9)-(norm.ppf(0.1))))


    capacity_mu = area_mu + powerdens_mu
    capacity_sigma = ((area_sigma**2)+(powerdens_sigma**2))**0.5
    eds = [lognorm.ppf(x/100, capacity_sigma, loc=0, scale=np.exp(capacity_mu)) for x in range(0,100)]
    indx = list(np.arange(1,101)[::-1])
    #edsepc_tups = list(zip(indx,eds))
    #prob_df = pd.DataFrame(edsepc_tups, columns = ['Cumulative confidence (%)', 'expected development size (MW)'])

    return indx, eds 

In [12]:
index, estimate = calculate_cumulative_conf(2, 8, 10, 15)
print(index, estimate)

[100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1] [0.0, 13.205802809105366, 15.398540422404356, 16.97492745638696, 18.26627158707802, 19.388798971870724, 20.398429154711877, 21.32684404003433, 22.193928613581523, 23.01307999246934, 23.79380841683307, 24.543142684249247, 25.26644756481281, 25.967927359395958, 26.65095080535273, 27.318268857382407, 27.97216540859641, 28.614564484255748, 29.247108295776485, 29.871215259155477, 30.488123913094448, 31.098926708651156, 31.70459639044439, 32.30600687092003, 32.90394995178081, 33.49914887311694, 34.09226941115389, 34.683929062098905, 35.27470471800395, 35.86513914490493, 36.45574650307342, 37.04701709678195